# Preprocess text

In [21]:
import io
from collections import defaultdict
import os, os.path
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd

import spacy
from spacy.lemmatizer import Lemmatizer
import nl_core_news_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import spacy
import enchant
from enchant.checker import SpellChecker

from string import punctuation

import torch
import re
import nltk

from difflib import SequenceMatcher
from transformers import AutoTokenizer, AutoModel

from src import iterators


#### Settings

In [22]:
#stopword_list = nltk.corpus.stopwords.words('dutch')
from spacy.lang.nl.stop_words import STOP_WORDS
limit = 0 # limit the data
stopwords = set(STOP_WORDS) # set of stopwords for performance

#### Import the data

In [23]:
# Load csv
csv = iterators.iterate_directory("../data/processed/selected_articles/", ".csv")
df = pd.concat([pd.read_csv(c["article_path"]) for c in csv],ignore_index=True)
df.sort_values(by=["count"], ascending=False, inplace=True)

### Clean the dataset

#### Remove non Chinese, Japanese and Korean char

In [24]:
def jkc_detect(texts):
    # korean
    if re.search("[\uac00-\ud7a3]", texts):
        return False
    # japanese
    if re.search("[\u3040-\u30ff]", texts):
        return False
    # chinese
    if re.search("[\u4e00-\u9FFF]", texts):
        return False
    return True

In [25]:
# Only non jkc
df['non_jkc'] = df['text'].apply(jkc_detect)
df = df[df['non_jkc'] == True]

In [26]:
df.tail(3)

,Unnamed: 0_x,type,text,article_name,date,index_article,article_filepath,dir,Unnamed: 0_y,metadata_title,...,newspaper_title,newspaper_date,newspaper_city,newspaper_publisher,newspaper_source,newspaper_volume,newspaper_issuenumber,newspaper_language,count,non_jkc
2516,129590,p,--—————— — ' ' Van harte geluk gewenst OU Part...,DDD_010850933_0074_articletext.xml,1950-02-08,147528,../data/1950/02-08/DDD_010850933/DDD_010850933...,../data/1950/02-08/DDD_010850933,139.0,DDD:ddd:010850933:mpeg21.didl.xml.gz.xml,...,De waarheid,1950-02-08,Amsterdam,s.n.,Internationaal Instituut voor Sociale Geschied...,9.0,235.0,nl,0,True
2518,49694,p,! Nieuwe ontdekking ' ! i behoedt UW meubelen ...,DDD_110585156_0045_articletext.xml,1950-09-25,118470,../data/1950/09-25/DDD_110585156/DDD_110585156...,../data/1950/09-25/DDD_110585156,918.0,DDD:ddd:110585156:mpeg21.didl.xml.gz.xml,...,De Telegraaf,1950-09-25,Amsterdam,Dagblad De Telegraaf,KB C 98,53.0,19635.0,nl,0,True
2517,41222,p,W.J. NiJBOER KLEERMAKERIJ TAMBLQNGWEG 28 wenst...,DDD_010896645_0029_articletext.xml,1950-12-31,115413,../data/1950/12-31/DDD_010896645/DDD_010896645...,../data/1950/12-31/DDD_010896645,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,True


#### Create functions for preprocessing

In [27]:
# Keep "." "!" and "?" to define end of sentence.

punctuation = ",/<>;':\"[]\\{}|`~@#$%^&*()_+-="

def remove_punctuation(text):
    """Remove punctuation"""
    no_punct = "".join([c for c in text if c not in punctuation])
    return(no_punct)

In [28]:
df["text_clean"] = df["text"].apply(lambda x: remove_punctuation(x))

In [29]:
def remove_stopwords(text):
    """Remove stopwords as defined by Spacy stopwords"""
    words = "".join([w for w in text if w not in stopwords])
    return words

In [10]:
df["text_clean"] = df["text_clean"].apply(lambda x: remove_stopwords(x))

In [30]:
def remove_numeric(text):
    """Remove numbers"""
    words = ''.join([c for c in text if not c.isdigit()])
    return words

In [31]:
df["text_clean"] = df["text_clean"].apply(lambda x: remove_numeric(x))

In [32]:
df["text_clean"][1]

'ning tijdens de ontgassing normaal kan doorgaan. In de Belgische mijn „Le Grand Trait te Frameries in Henegouwen „oogstte men op deze wijze in  maanden tijds . m  methaangas in de mijn „Saint Albert te Ressaix in een iets langere periode . m  methaan. In Henegouwen wordt het gas reeds naar buiten geleverd via de lichtgasfabrieken te Tertre. Methaangas levert  tot  caloriën warmte hetgeen tweemaal zoveel is als gewoon cokesovengas. In vele andere mijnen waaronder de Kempische neemt men proeven. Er bestaan plannen in Belgisch Limburg een leidingermet aan te leggen voor de distributie van het gas aan de bevolking. Een probleem vormt echter de vrij onregelmatige toevoer waarmee men ongetwijfeld te kampen zal krijgen. In de mijn Hirschbach in het Saargebied heeft men een andere methode gevolgd. In deze mijn ontsnapte zoveel gas dat met luchtverversing niet voldoende te bereiken was. Een gedeelte van de mijn werd daarom met dammen van de rest afgesloten. Door hel verrichten van boringen en 

In [33]:
def cleaner(df):
    "Extract relevant text from DataFrame using a regex"
    # Regex pattern for only alphanumeric, hyphenated text with 3 or more chars
    pattern = re.compile(r"[A-Za-z\-]{2,40}")
    df['text_no_point'] = df['text_clean'].str.findall(pattern).str.join(' ')
    if limit > 0:
        return df.iloc[:limit, :].copy()
    else:
        return df

In [34]:
cleaner(df)

,Unnamed: 0_x,type,text,article_name,date,index_article,article_filepath,dir,Unnamed: 0_y,metadata_title,...,newspaper_city,newspaper_publisher,newspaper_source,newspaper_volume,newspaper_issuenumber,newspaper_language,count,non_jkc,text_clean,text_no_point
0,36288,p,Aoiang er mijnen bestaan is het mijngas de gro...,DDD_010417712_0100_articletext.xml,1950-11-16,113601,../data/1950/11-16/DDD_010417712/DDD_010417712...,../data/1950/11-16/DDD_010417712,882.0,DDD:ddd:010417712:mpeg21.didl.xml.gz.xml,...,Heerlen,Nieuwe Limburger koerier;Uitgeversmaatschappĳ ...,Sociaal Historisch centrum voor Limburg T 501,33.0,269.0,nl,28,True,Aoiang er mijnen bestaan is het mijngas de gro...,Aoiang er mijnen bestaan is het mijngas de gro...
1,36381,p,ning tijdens de ontgassing normaal kan doorgaa...,DDD_010417712_0102_articletext.xml,1950-11-16,113628,../data/1950/11-16/DDD_010417712/DDD_010417712...,../data/1950/11-16/DDD_010417712,882.0,DDD:ddd:010417712:mpeg21.didl.xml.gz.xml,...,Heerlen,Nieuwe Limburger koerier;Uitgeversmaatschappĳ ...,Sociaal Historisch centrum voor Limburg T 501,33.0,269.0,nl,20,True,ning tijdens de ontgassing normaal kan doorgaa...,ning tijdens de ontgassing normaal kan doorgaa...
2,107454,p,"""W/ij eijn deze keer op een Joodse bruiloft, ""...",DDD_010612570_0079_articletext.xml,1950-01-28,139421,../data/1950/01-28/DDD_010612570/DDD_010612570...,../data/1950/01-28/DDD_010612570,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15,True,Wij eijn deze keer op een Joodse bruiloft die...,Wij eijn deze keer op een Joodse bruiloft die ...
3,122625,p,Het is een spannende geschiedenis met de gasvo...,DDD_010417601_0094_articletext.xml,1950-06-07,144892,../data/1950/06-07/DDD_010417601/DDD_010417601...,../data/1950/06-07/DDD_010417601,115.0,DDD:ddd:010417601:mpeg21.didl.xml.gz.xml,...,Heerlen,Nieuwe Limburger koerier;Uitgeversmaatschappĳ ...,Sociaal Historisch centrum voor Limburg T 501,33.0,132.0,nl,14,True,Het is een spannende geschiedenis met de gasvo...,Het is een spannende geschiedenis met de gasvo...
2519,125000,p,"In elk geval, meende de archivaris, heeft pate...",DDD_011199673_0059_articletext.xml,1950-01-10,145756,../data/1950/01-10/DDD_011199673/DDD_011199673...,../data/1950/01-10/DDD_011199673,123.0,DDD:ddd:011199673:mpeg21.didl.xml.gz.xml,...,'s-Hertogenbosch,Gebr. Verhoeven,Koninklijke Bibliotheek C 236,105.0,34325.0,nl,13,True,In elk geval meende de archivaris heeft pater ...,In elk geval meende de archivaris heeft pater ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,49598,p,Kamperveen kreeg het in zijn hoofd binnen het ...,DDD_010480643_0050_articletext.xml,1950-09-25,118429,../data/1950/09-25/DDD_010480643/DDD_010480643...,../data/1950/09-25/DDD_010480643,917.0,DDD:ddd:010480643:mpeg21.didl.xml.gz.xml,...,Paramaribo,A.J. Morpurgo,KB C 197,7.0,2174.0,nl,1,True,Kamperveen kreeg het in zijn hoofd binnen het ...,Kamperveen kreeg het in zijn hoofd binnen het ...
2377,42812,p,Toen het eilander busje met zijn interieur van...,DDD_011202192_0180_articletext.xml,1950-12-09,116042,../data/1950/12-09/DDD_011202192/DDD_011202192...,../data/1950/12-09/DDD_011202192,900.0,DDD:ddd:011202192:mpeg21.didl.xml.gz.xml,...,'s-Hertogenbosch,Gebr. Verhoeven,Koninklijke Bibliotheek C 236,106.0,34603.0,nl,1,True,Toen het eilander busje met zijn interieur van...,Toen het eilander busje met zijn interieur van...
2516,129590,p,--—————— — ' ' Van harte geluk gewenst OU Part...,DDD_010850933_0074_articletext.xml,1950-02-08,147528,../data/1950/02-08/DDD_010850933/DDD_010850933...,../data/1950/02-08/DDD_010850933,139.0,DDD:ddd:010850933:mpeg21.didl.xml.gz.xml,...,Amsterdam,s.n.,Internationaal Instituut voor Sociale Geschied...,9.0,235.0,nl,0,True,—————— — Van harte geluk gewenst OU Partu n ...,Van harte geluk gewenst OU Partu krant geluk g...
2518,49694,p,! Nieuwe ontdekking ' ! i behoedt UW meubelen ...,DDD_110585156_0045_articletext.xml,1950-09-25,118470,../data/1950/09-25/DDD_110585156/DDD_110585156...,../data

### Split long text

In [35]:
def get_split(txt, length):
  len_tot = []
  len_partial = []
  if len(txt.split())//length >0:
    n = len(txt.split())//length
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      len_partial = txt.split()[:length]
      len_tot.append(" ".join(len_partial))
    else:
      len_partial = txt.split()[w*length:w*length + length]
      len_tot.append(" ".join(len_partial))
  return len_tot

In [36]:
df["text_clean_split"] = df["text_clean"].apply(get_split, length=500)
df["text_clean_split_no_point"] = df["text_no_point"].apply(get_split, length=500)

### Save cleaned df

In [37]:
df.to_csv("cleaned_df.csv")

----

### Additional subsetting only to relevant part

Select the divided text and retrieve also article_ids and article_name

In [154]:
divided_texts = []
idx_texts = []
name_texts = []
dfids_texts = []
for idx, row in df.iterrows():
  for text in row['text_split']:
    divided_texts.append(text)
    idx_texts.append(idx)
    name_texts.append(row["article_name"])
    dfids_texts.append(row["Unnamed: 0_x"])

Create smaller dataframe for analysis

In [155]:
df_texts = pd.DataFrame({"text":divided_texts, "article_id":dfids_texts, "article_name":name_texts})
df_texts.head(5)

,text,article_id,article_name
0,Aoiang er mijnen bestaan is het mijngas de gro...,36288,DDD_010417712_0100_articletext.xml
1,ning tijdens de ontgassing normaal kan doorgaa...,36381,DDD_010417712_0102_articletext.xml
2,"""W/ij eijn deze keer op een Joodse bruiloft, ""...",107454,DDD_010612570_0079_articletext.xml
3,Het is een spannende geschiedenis met de gasvo...,122625,DDD_010417601_0094_articletext.xml
4,"In elk geval, meende de archivaris, heeft pate...",125000,DDD_011199673_0059_articletext.xml
